In [103]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.cross_decomposition import PLSRegression
from sklearn.naive_bayes import GaussianNB
import xgboost as xgb
import calendar
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
import math

In [74]:
ml_csv = pd.read_csv('dsci_power_outage_data.csv')

In [20]:
ml_csv.head(10)

,Delay_Hours,Month,System Voltage,Consumer_Effected,System_Load (kW),Failure_Code,Location
0,0.10,January,33kV,9555,23290.0,A1,Location 1
1,0.42,January,LV,282,98.0,D1,Location 2
2,0.50,January,11kV,291,104.0,C1,Location 3
3,0.08,January,11kV,4192,13025.0,C1,Location 2
4,0.15,January,11kV,755,807.0,C1,Location 4
5,0.33,January,11kV,755,367.0,C1,Location 4
6,0.53,January,11kV,697,217.0,C1,Location 7
7,0.50,January,LV,192,192.0,D2,Location 5
8,0.02,January,11kV,10682,127700.0,C1,Location 5
9,0.22,January,11kV,3365,6336.0,B4,Location 6


In [16]:
print("Verifiyng Data Types")
print(ml_csv.dtypes)

Verifiyng Data Types
Delay_Hours          float64
Month                 object
System Voltage        object
Consumer_Effected      int64
System_Load (kW)     float64
Failure_Code          object
Location              object
dtype: object


In [28]:
#check for nulls
print("Checking for nulls")
print("Delay_Hours")
print(ml_csv['Delay_Hours'].isnull().values.any())
print("Month")
print(ml_csv['Month'].isnull().values.any())
print("System Voltage")
print(ml_csv['System Voltage'].isnull().values.any())
print("Consumer_Effected")
print(ml_csv['Consumer_Effected'].isnull().values.any())
print("System_Load (kW)")
print(ml_csv['System_Load (kW)'].isnull().values.any())
print("Failure_Code")
print(ml_csv['Failure_Code'].isnull().values.any())
print("Location")
print(ml_csv['Location'].isnull().values.any())

#check for duplicates
print("\n")
print("Checking for duplicate rows")
print(ml_csv.duplicated().any())
print("\n")
print("Verifiyng Data Types")
print(ml_csv.dtypes)

print("\n")
print(len(ml_csv))
print(len(ml_csv.dropna()))
print("\n")
print(len(ml_csv.drop_duplicates(keep="last")))

Checking for nulls
Delay_Hours
True
Month
True
System Voltage
True
Consumer_Effected
False
System_Load (kW)
True
Failure_Code
False
Location
True


Checking for duplicate rows
True


Verifiyng Data Types
Delay_Hours          float64
Month                 object
System Voltage        object
Consumer_Effected      int64
System_Load (kW)     float64
Failure_Code          object
Location              object
dtype: object


5385
5376


4247


In [92]:
ml_csv = (ml_csv.dropna())


In [93]:
fig = px.scatter(ml_csv, 
              x = ml_csv.index.values, 
              y = 'Delay_Hours',
              title = 'Delay_Hours by index'
            )

fig.show()

In [46]:
ml_csv_group = ml_csv
ml_csv_group['countVal'] = 1
ml_csv_group = ml_csv_group.groupby(['Month', 'Location', 'Failure_Code']).agg(
        Delay_Hours=('Delay_Hours', 'sum')
    ).reset_index()
ml_csv_group

,Month,Location,Failure_Code,Delay_Hours
0,April,Location 1,A1,10.96
1,April,Location 1,B1,0.92
2,April,Location 1,B4,0.25
3,April,Location 1,C1,9.90
4,April,Location 1,C2,0.28
...,...,...,...,...
519,September,Location 7,C1,6.13
520,September,Location 7,C2,2.19
521,September,Location 7,C5,0.34
522,September,Location 7,D1,1.50


In [47]:
fig = px.scatter(ml_csv_group, 
              x = ml_csv_group.index.values, 
              y = 'Delay_Hours',
              title = 'Delay_Hours by index'
            )

fig.show()

In [ ]:
fig = px.bar(ml_csv_group,
            x="Month", 
            y="Delay_Hours", 
            color="Location", 
            title="Delay Hours Monthly by Location"
          )

fig.show()

In [128]:
fig = px.bar(ml_csv_group,
            x="Month", 
            y="Delay_Hours", 
            color="Failure_Code", 
            title="Delay Hours Monthly by Month"
          )

fig.show()

In [51]:
fig = px.bar(ml_csv_group,
            x="Location", 
            y="Delay_Hours", 
            color="Failure_Code", 
            title="Delay Hours Monthly by Location"
          )

fig.show()

In [84]:
# https://stackoverflow.com/questions/3418050/how-to-map-month-name-to-month-number-and-vice-versa
monthTransform = {month: index for index, month in enumerate(calendar.month_name) if month}
voltageTransform = {'LV': 1, '11kV': 2, '33kV': 3}

In [94]:
ml_csv['Month_Number'] = ml_csv['Month'].map(monthTransform)
ml_csv['System_Voltage'] = ml_csv['System Voltage'].map(voltageTransform)
ml_csv['Location'] = ml_csv['Location'].str.replace('Location ', '', regex=False)
#ml_csv['Failure_Code_Number'] = int(ml_csv['Failure_Code'], 16)


C:\Users\thoma\AppData\Local\Temp\ipykernel_7356\2540232366.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\thoma\AppData\Local\Temp\ipykernel_7356\2540232366.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\thoma\AppData\Local\Temp\ipykernel_7356\2540232366.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

In [95]:
ml_csv

,Delay_Hours,Month,System Voltage,Consumer_Effected,System_Load (kW),Failure_Code,Location,Month_Number,System_Voltage
0,0.10,January,33kV,9555,23290.0,A1,1,1,3
1,0.42,January,LV,282,98.0,D1,2,1,1
2,0.50,January,11kV,291,104.0,C1,3,1,2
3,0.08,January,11kV,4192,13025.0,C1,2,1,2
4,0.15,January,11kV,755,807.0,C1,4,1,2
...,...,...,...,...,...,...,...,...,...
5379,0.02,December,33kV,12368,180250.0,A1,1,12,3
5380,0.02,December,33kV,12368,180250.0,A1,1,12,3
5381,0.02,December,33kV,24039,196450.0,A1,1,12,3
5382,0.02,December,33kV,24039,196450.0,A1,1,12,3


In [137]:
X = ml_csv[["Month_Number", "Location", "System_Voltage",]]
y = ml_csv["Delay_Hours"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)

In [138]:
pls2 = PLSRegression(n_components=2)

pls2.fit(X, y)

PLSRegression()

Y_pred = pls2.predict(X)

In [139]:
rmse = math.sqrt(mean_squared_error(y, Y_pred))
rmse

1.6771970284979383

In [112]:
gnb = GaussianNB()
y_pred = gnb.fit(X_train, y_train).predict(X_test)

ValueError: Unknown label type: (array([2.000e-02, 3.000e-02, 5.000e-02, 7.000e-02, 8.000e-02, 1.000e-01,
       1.200e-01, 1.300e-01, 1.500e-01, 1.700e-01, 1.800e-01, 2.000e-01,
       2.200e-01, 2.300e-01, 2.500e-01, 2.700e-01, 2.800e-01, 3.000e-01,
       3.200e-01, 3.300e-01, 3.500e-01, 3.700e-01, 3.800e-01, 4.000e-01,
       4.200e-01, 4.300e-01, 4.500e-01, 4.700e-01, 4.800e-01, 5.000e-01,
       5.200e-01, 5.300e-01, 5.500e-01, 5.700e-01, 5.800e-01, 6.000e-01,
       6.200e-01, 6.300e-01, 6.500e-01, 6.700e-01, 6.800e-01, 7.000e-01,
       7.200e-01, 7.300e-01, 7.500e-01, 7.700e-01, 7.800e-01, 8.000e-01,
       8.200e-01, 8.300e-01, 8.500e-01, 8.700e-01, 8.800e-01, 9.000e-01,
       9.200e-01, 9.300e-01, 9.500e-01, 9.700e-01, 9.800e-01, 1.000e+00,
       1.020e+00, 1.030e+00, 1.050e+00, 1.070e+00, 1.080e+00, 1.100e+00,
       1.120e+00, 1.130e+00, 1.150e+00, 1.170e+00, 1.180e+00, 1.200e+00,
       1.220e+00, 1.230e+00, 1.250e+00, 1.270e+00, 1.280e+00, 1.320e+00,
       1.330e+00, 

In [114]:
X = ml_csv["Month_Number"]
y = ml_csv["Delay_Hours"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)

In [136]:
clf = xgb.XGBModel(tree_method="hist", seed = 123, early_stopping_rounds=2)

clf.fit(X_train, y_train)

ValueError: Must have at least 1 validation dataset for early stopping.